# Lab 5: Data manipulation


In [2]:
require(tidyverse)
require(stringr)

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.1.0     ✔ purrr   0.3.0
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0

Warning message:
“package ‘tibble’ was built under R version 3.5.2”
Warning message:
“package ‘dplyr’ was built under R version 3.5.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Data Import

The package `readr` (part of `tidyverse`) contains several functions for reading in flat data.

`read_csv` reads standard comma-delimited files. There are variants like `read_csv2` (semicolon-delimited) and `read_tsv` (tab-delimited), while `read_delim` allows reading in files with any delimiter.

Also note that the equivalents for writing/saving data files also exist, called `write_csv`, etc.

Note that these are variants of `read.table`, `read.csv`, `write.table`, etc. which are a part of base R and can also be used for reading in files.

In [3]:
mtcars$car = rownames(mtcars)

write_csv(mtcars, "mtcars.csv")
mydat = read_csv("mtcars.csv")
head(mydat)

Parsed with column specification:
cols(
  mpg = col_double(),
  cyl = col_integer(),
  disp = col_double(),
  hp = col_integer(),
  drat = col_double(),
  wt = col_double(),
  qsec = col_double(),
  vs = col_integer(),
  am = col_integer(),
  gear = col_integer(),
  carb = col_integer(),
  car = col_character()
)



mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car
<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<chr>
21.0,6,160,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
21.0,6,160,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
22.8,4,108,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
18.7,8,360,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
18.1,6,225,105,2.76,3.460,20.22,1,0,3,1,Valiant


Notice that `readr` gets its column names from the first row of the CSV. If the first lines are metadata instead of column names, you can use the following to skip those lines.

In [12]:
read_csv(
"The first line of metadata
The second line of metadata
x,y,z
1,2,3", skip=2)

x,y,z
1,2,3


In [13]:
read_csv(
"# A comment I want to skip
x,y,z
1,2,3", comment='#')

x,y,z
1,2,3


Sometimes the columns don't have names. Then you can tell `readr` to create the column names.

In [14]:
read_csv("mtcars.csv", col_names=FALSE)

X1,X2,X3
1,2,3
4,5,6


In [15]:
read_csv(
"1,2,3
4,5,6", col_names=c("x", "y", "z"))

x,y,z
1,2,3
4,5,6


Another option that commonly needs tweaking is `na`: this specifies the value (or values) that are used to represent missing values in your file:

In [16]:
read_csv(
"a,b,c
1,2,.", na='.')

a,b,c
1,2,NA


## `gather` and `spread`

Remember that `gather` transforms datasets from "wide" to "long," collecting different column names into a variable. `spread` does the opposite, turning "long" datasets into "wide" ones by spreading out a variable's values into the column names.

In [5]:
grades_wide = tribble(
  ~name,  ~`2015`, ~`2016`, ~`2017`,
'Roger',       83,      89,      93,
  'Jon',       92,      90,      93)
grades_wide

name,2015,2016,2017
<chr>,<dbl>,<dbl>,<dbl>
Roger,83,89,93
Jon,92,90,93


In [6]:
grades_long = grades_wide %>% gather(key=year, value=grade, `2015`:`2017`)
grades_long

name,year,grade
<chr>,<chr>,<dbl>
Roger,2015,83
Jon,2015,92
Roger,2016,89
Jon,2016,90
Roger,2017,93
Jon,2017,93


In [19]:
grades_long %>% spread(key=year, value=grade)

name,2015,2016,2017
Jon,92,90,93
Roger,83,89,93


## MLB Data

This dataset contains information for player-seasons in the American League from 2015 to 2018. For those unfamiliar with baseball:
* `PA`: Plate Appearances, the number of times a player came up to bat.
* `HR`: number of home runs.
* `BBrate`: The number of walks (BBs) as a percentage of plate appearances.
* `BB`: The number of walks.
* `K`: The number of strikeouts.
* `AVG`: A batters batting average.
* `FB`: The number of fly balls a batter hit.

Note: Also recall the functions `unite`, `separate`, and `complete`.

In [7]:
mlb = read_csv('mlb.csv')
head(mlb)

Parsed with column specification:
cols(
  year = col_integer(),
  name = col_character(),
  team = col_character(),
  division = col_character(),
  PA = col_integer(),
  HR = col_integer(),
  BBrate = col_character(),
  BB_K = col_character(),
  AVG = col_double(),
  FB = col_integer(),
  playerid = col_integer()
)



year,name,team,division,PA,HR,BBrate,BB_K,AVG,FB,playerid
<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<int>
2016,Abraham Almonte,Indians,Central,194,1,4.1 %,8/42,0.264,43,5486
2017,Abraham Almonte,Indians,Central,195,3,10.3 %,20/46,0.233,34,5486
2015,Adam Moore,Indians,Central,4,0,0.0 %,0/2,0.250,1,9362
2016,Adam Moore,Indians,Central,5,0,0.0 %,0/4,0.000,1,9362
2018,Adam Plutko,Indians,Central,2,0,0.0 %,0/0,0.000,0,15846
2018,Adam Rosales,Indians,Central,21,1,4.8 %,1/5,0.211,7,9682


### Problem 1

Note that `BBrate` and `BB_K` were read in as strings. Clean up these variables and convert them to numeric variables, noting that `BB_K` should be two variables named `BB` and `K`.

In [1]:
dat1 = NA
head(dat1)

[1] NA

### Problem 2

Calculate the HR per FB rate for each team and year. Convert this to a long dataset, so your variables should be `division`, `team`, and `2015`-`2018`, where values are the HR/FB rate. Note that you should ensure that `division` is still in the dataset.

Create a variable called `increased`, which checks if the HR/FB rate was higher in 2018 than it was in 2015 for that team.

In [2]:
dat2 = NA

head(dat2)

[1] NA

Turn this back into a "long" dataset and create a plot of HR/FB rate on year. Color it by `team`, facet it by `division`, and choose the linetype according to the `increased` variable.

In [7]:
dat2b = dat2


### Problem 3

Go back to `mlb` and calculate total HRs and PAs per year per team. Create a wide version of this dataset. So there should be a `team` variable, then eight variables tracking values: `HR_2015`, `HR_2016`, `HR_2017`, `HR_2018`, `PA_2015`, `PA_2016`, `PA_2017`, and `PA_2018`.

Hint: Once you calculate the values you can use a `gather`, `unite`, and `spread` in that order to create the wide version.

In [8]:
dat3 = NA

head(dat3)

[1] NA

### Problem 4

Remove player-seasons with less than 200 `PA`. Then, for each player, calculate the change in `AVG` from the previous year to the current year and the change from the current year to the next year. Store these as two variables. If data on the previous or next year is not present, then the corresponding variable should be `NA`.

Once you have these two variables, make two plots where the `x`-axis is current year `AVG` and the `y`-axis is each of these difference variables. Can you make both plots with a single command using faceting? (Hint: you may need to do some additional data transformations to accomplish this.) What conclusions can you draw from these plots?

In [10]:
dat4 = NA

head(dat4)

[1] NA